Importing Libraries 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from transformers import DistilBertTokenizer, DistilBertModel
from sentence_transformers import SentenceTransformer
import xgboost


d:\Anaconda\envs\fakenews_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download NLTK

In [2]:
# nltk_data_path = "D:/FakeNewsDetection/nltk_data"
# nltk.data.path.append(nltk_data_path)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Ensure the stopwords are loaded from the correct directory
stop_words = set(stopwords.words('english', ))
punctuation = set(string.punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hello\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hello\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\hello\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


PreProcess

In [3]:
def preprocess_text(row):
    # combine title, subject, and text into one string
    combined_text = f"Title: {row['title']} | Subject: {row['subject']} | Text: {row['text']} "
    # convert to lowercase
    text = combined_text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # 
    tokens = [word for word in tokens if word not in punctuation and word not in stop_words]
    # 
    return ' '.join(tokens)

Importing DataSet

In [4]:
df_true = pd.read_csv('../data/true.csv')
df_fake = pd.read_csv('../data/fake.csv')

In [5]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Inserting column 'label' 

In [7]:
df_true['label'] = 1 # 1 for true news
df_fake['label'] = 0 # 0 for fake news
# Concatenate the two dataframes
df = pd.concat([df_true, df_fake], axis=0, ignore_index=True)
#spilt 80% of the data for training and 20% for testing
#stratify the split by label to ensure both classes are represented in both sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

train_data['processed_text'] = train_data.apply(preprocess_text, axis=1)
test_data['processed_text'] = test_data.apply(preprocess_text, axis=1)



Embedding


In [8]:

from tqdm import tqdm

# Load mô hình BERT nhẹ (paraphrase-MiniLM-L6-v2)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Hàm tạo embedding theo batch (Không cần torch)
def get_batch_embeddings(text_list, batch_size=16):
    embeddings = []
    for i in tqdm(range(0, len(text_list), batch_size), desc="Embedding..."):
        batch_texts = text_list[i:i+batch_size]
        batch_embeddings = model.encode(batch_texts)  # Trả về numpy array
        embeddings.extend(batch_embeddings)

    return np.array(embeddings)

# Chuyển train và test thành embedding
# X_train = get_batch_embeddings(train_data['processed_text'].tolist(), batch_size=16)
# X_test = get_batch_embeddings(test_data['processed_text'].tolist(), batch_size=16)

# Lấy nhãn
y_train = train_data['label'].values
y_test = test_data['label'].values

# Kiểm tra kích thước đầu ra
# print("X_train shape:", X_train.shape)  # (32000, 384) nếu có 40.000 tin tức
# print("X_test shape:", X_test.shape)    # (8000, 384)


In [9]:
# np.save('../embedding/X_train_embedding.npy', X_train)
# np.save('../embedding/X_test_embedding.npy', X_test)

# print("Embedding đã được lưu vào file .npy")

In [12]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import xgboost as xgb

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6],
    'learning_rate': [0.05, 0.1]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
xgb_model = xgb.XGBClassifier(random_state=42)

grid_search = GridSearchCV(xgb_model, param_grid, cv=cv, scoring='accuracy', n_jobs=1, verbose=2)
grid_search.fit(x_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)



Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=  14.8s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=  13.2s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=100; total time=  13.0s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=  29.3s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=  30.1s
[CV] END ..learning_rate=0.05, max_depth=4, n_estimators=200; total time=  25.2s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=100; total time=  20.3s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=100; total time=  20.0s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=100; total time=  21.0s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  40.8s
[CV] END ..learning_rate=0.05, max_depth=6, n_estimators=200; total time=  37.8s
[CV] END ..learning_rate=0.05, max_depth=6, n_est

ML Training

In [11]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

x_train = np.load('../embedding/X_train_embedding.npy')
x_test = np.load('../embedding/X_test_embedding.npy')

# clf = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
# clf.fit(x_train, y_train)

# y_pred = clf.predict(x_test)
# print("Accuracy:", accuracy_score(y_test, y_pred))
